In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use(['dark_background'])

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from mlxtend.plotting import plot_decision_regions

from sklearn.feature_selection import SelectKBest, chi2
pd.options.display.max_columns = 680
from functions_pkg import print_vif, predictions_df

In [2]:
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/epi_r.csv')

### EDA & Cleaning

In [3]:
df = df.drop(columns=["title"])

In [4]:
df.isna().mean().sort_values(ascending=False).head(10)

fat               0.208608
protein           0.207560
sodium            0.205416
calories          0.205316
turkey            0.000000
fourth of july    0.000000
fritter           0.000000
frittata          0.000000
friendsgiving     0.000000
freezer food      0.000000
dtype: float64

In [5]:
df = df.dropna(subset=['fat', 'protein', 'sodium', 'calories'])
df.head(5)

,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,30 days of groceries,advance prep required,alabama,alaska,alcoholic,almond,amaretto,anchovy,anise,anniversary,anthony bourdain,aperitif,appetizer,apple,apple juice,apricot,arizona,artichoke,arugula,asian pear,asparagus,aspen,atlanta,australia,avocado,back to school,backyard bbq,bacon,bake,banana,barley,basil,bass,bastille day,bean,beef,beef rib,beef shank,beef tenderloin,beer,beet,bell pepper,berry,beverly hills,birthday,biscuit,bitters,blackberry,blender,blue cheese,blueberry,boil,bok choy,bon appétit,bon app��tit,boston,bourbon,braise,bran,brandy,bread,breadcrumbs,breakfast,brie,brine,brisket,broccoli,broccoli rabe,broil,brooklyn,brown rice,brownie,brunch,brussel sprout,buffalo,buffet,bulgaria,bulgur,burrito,butter,buttermilk,butternut squash,butterscotch/caramel,cabbage,cake,california,calvados,cambridge,campari,camping,canada,candy,candy thermometer,cantaloupe,capers,caraway,cardamom,carrot,cashew,casserole/gratin,cauliflower,caviar,celery,chambord,champagne,chard,chartreuse,cheddar,cheese,cherry,chestnut,chicago,chicken,chickpea,chile,chile pepper,chili,chill,chive,chocolate,christmas,christmas eve,cilantro,cinco de mayo,cinnamon,citrus,clam,clove,cobbler/crumble,cocktail,cocktail party,coconut,cod,coffee,coffee grinder,cognac/armagnac,collard greens,colorado,columbus,condiment,condiment/spread,connecticut,cook like a diner,cookbook critic,cookie,cookies,coriander,corn,cornmeal,costa mesa,cottage cheese,couscous,crab,cranberry,cranberry sauce,cream cheese,créme de cacao,crêpe,cr��me de cacao,cuba,cucumber,cumin,cupcake,currant,curry,custard,dairy,dairy free,dallas,date,deep-fry,denver,dessert,digestif,dill,dinner,dip,diwali,dominican republic,dorie greenspan,double boiler,dried fruit,drink,drinks,duck,easter,eau de vie,edible gift,egg,egg nog,eggplant,egypt,emeril lagasse,endive,engagement party,england,entertaining,epi + ushg,epi loves the microwave,escarole,fall,family reunion,fat free,father's day,fennel,feta,fig,fish,flaming hot summer,flat bread,florida,fontina,food processor,fortified wine,fourth of july,france,frangelico,frankenrecipe,freeze/chill,freezer food,friendsgiving,frittata,fritter,frozen dessert,fruit,fruit juice,fry,game,garlic,georgia,germany,gin,ginger,goat cheese,goose,gouda,gourmet,graduation,grains,grand marnier,granola,grape,grapefruit,grappa,green bean,green onion/scallion,grill,grill/barbecue,ground beef,ground lamb,guam,guava,haiti,halibut,halloween,ham,hamburger,hanukkah,harpercollins,hawaii,hazelnut,healdsburg,healthy,herb,high fiber,hollywood,hominy/cornmeal/masa,honey,honeydew,hors d'oeuvre,horseradish,hot drink,hot pepper,house & garden,house cocktail,houston,hummus,ice cream,ice cream machine,iced coffee,iced tea,idaho,illinois,indiana,iowa,ireland,israel,italy,jalapeño,jam or jelly,jamaica,japan,jerusalem artichoke,juicer,jícama,kahlúa,kale,kansas,kansas city,kentucky,kentucky derby,kid-friendly,kidney friendly,kirsch,kitchen olympics,kiwi,kosher,kosher for passover,kumquat,kwanzaa,labor day,lamb,lamb chop,lamb shank,lancaster,las vegas,lasagna,leafy green,leek,legume,lemon,lemon juice,lemongrass,lentil,lettuce,lima bean,lime,lime juice,lingonberry,liqueur,lobster,london,long beach,los angeles,louisiana,louisville,low cal,low carb,low cholesterol,low fat,low sodium,low sugar,low/no sugar,lunar new year,lunch,lychee,macadamia nut,macaroni and cheese,maine,mandoline,mango,maple syrup,mardi gras,margarita,marinade,marinate,marsala,marscarpone,marshmallow,martini,maryland,massachusetts,mayonnaise,meat,meatball,meatloaf,melon,mexico,mezcal,miami,michigan,microwave,midori,milk/cream,minneapolis,minnesota,mint,mississippi,missouri,mixer,molasses,monterey jack,mortar and pestle,mother's day,mozzarella,muffin,mushroom,mussel,mustard,mustard greens,nancy silverton,nebraska,nectarine,new hampshire,new jersey,new mexico,new orleans,new year's day,new year's eve,new york,"no meat, no problem",no sugar added,n

### Feature Selection

> SelectKBest

In [6]:
skb = SelectKBest(score_func=chi2, k=30)

In [7]:
bin_cols = df.drop(columns=["rating", "calories", "protein", "fat", "sodium"])
target = df.rating > 4
target.shape

(15864,)

> Print VIF

In [8]:
skb.fit(bin_cols, target)
kb_cols = df[bin_cols.columns[skb.get_support()]]

print_vif(kb_cols)

VIF results
-------------------------------
const                4.010122
alcoholic            4.701492
backyard bbq         1.553148
bitters              1.194254
bon appétit          1.069325
christmas            1.091630
cocktail             1.582135
cocktail party       1.502226
cranberry            1.087498
dinner               1.183258
drink                3.782851
fall                 1.184963
gin                  1.419664
goat cheese          1.008397
grill                1.821417
grill/barbecue       1.664225
harpercollins        1.015141
house & garden       1.351249
low carb             1.028867
pasta                1.035600
peanut free          9.269596
quick & easy         1.060789
roast                1.077682
rum                  1.143894
soy free             9.044296
spirit               1.406468
stuffing/dressing    1.091628
thanksgiving         1.497918
tofu                 1.022900
vegan                1.079952
turkey               1.142917
dtype: float64
-----------

In [9]:
df = df[list(bin_cols.columns[skb.get_support()]) + ['protein', 'sodium', 'fat', 'calories','rating']]

> Changing rating to binary

In [10]:
# change rating feature to a binary var
df.loc[df.rating <= 4, "rating"] = 0
df.loc[df.rating > 4, "rating"] = 1

> ### testing sets

In [11]:
X = df.drop(columns=['rating'])
y = df.rating

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28)

> ### Preprocessing

In [13]:
preprocessing = ColumnTransformer(
    [
        ("scale", StandardScaler(), X.columns)
    ]
)

> ### Pipeline

In [14]:
pipeline = Pipeline(
    [
        ("preprocessing", preprocessing),
        ("model", SVC(random_state=28)),
    ]
)

In [15]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  Index(['alcoholic', 'backyard bbq', 'bitters', 'bon appétit', 'christmas',
       'cocktail', 'cocktail party', 'cranberry', 'dinner', 'drink', 'fall',
       'gin', 'goat cheese', 'grill', 'grill/barbecue', 'harpercollins',
       'house & garden', 'low carb', 'pasta', 'peanut free', 'quick & easy',
       'roast', 'rum', 'soy free', 'spirit', 'stuffing/dressing',
       'thanksgiving', 'tofu', 'vegan', 'turkey', 'protein', 'sodium', 'fat',
       'calories'],
      dtype='object'))])),
                ('model', SVC(random_state=28))])

In [16]:
print(pipeline.score(X_train, y_train))
print(pipeline.score(X_test, y_test))

0.5990859664329052
0.5843050740624015


> ### GridSearchCV()

In [17]:
grid = {
    "model__C": 
    [
#         0.1, 
        1.0, 
#         10.0
    ],
    "model__kernel": 
    [
#         'linear', 
        'rbf', 
#         'poly'
    ],
    "model__gamma": 
    [
        0.1, 
#         1
    ],
    "model__degree": 
    [
        2, 
#         3
    ]
}

pipeline_cv = GridSearchCV(pipeline, grid, cv=2, verbose=2, n_jobs=-1)
pipeline_cv.fit(X_train, y_train)

print("Best hyperparameters:")
pipeline_cv.best_params_

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   14.5s finished


Best hyperparameters:


{'model__C': 1.0,
 'model__degree': 2,
 'model__gamma': 0.1,
 'model__kernel': 'rbf'}

In [18]:
train_score = pipeline_cv.score(X_train, y_train)
test_score = pipeline_cv.score(X_test, y_test)

print(f"train_score: {train_score}")
print(f"test_score: {test_score}")

train_score: 0.6168150657946576
test_score: 0.579892845887173
